In [1]:
#   |'''''''''''''╔╬╬╬╬╬╬╬╬   _____  _____      _____  _____      ___   __
#   |            ╔╬╬╬╬╬╬╬╬╬  |\   _ \  _  \    |\   _ \  _  \    |\  \|\  \
#   | ░░         ╬╬╬╬╬╬╬╬╬╬  \ \  \\__\ \  \   \ \  \\__\ \  \   \ \  \/  /|_
#    ░░░░        ╬╬╬╬╬╬╬╬╬╬   \ \  \|__| \  \   \ \  \|__| \  \   \ \   ___  \
#   ░░░░░╦╬╦    ╔╬╬╬╬╬╬╬╬╬╬    \ \  \   \ \  \   \ \  \   \ \  \   \ \  \ \   \
#  ░░░░░╬╬╬╬ ▓▓└╬╬╬╬╬╬╬╬╬╬╬     \ \__\   \ \__\   \ \__\   \ \__\   \ \__\ \___\
# ░░░░░╔╬╬╬ ▓▓▓  ╓╬╬╬╬╬╬╬╬╬      \|__|    \|__|    \|__|    \|__|    \|__| \|__|
# ░░░░░╠╬╬╬ ▓▓▓  └╬╬╬╬╬╬╬╬╬
#  ░░░░└╬╬╬╬ ▓▓   ╬╬╬╬╬╬╬╬╬  Lehrstuhl für Mensch-Maschine-Kommunikation
#  ░░░░░╙╬╬╬╩            ╬╬  Technische Universität München
#   ░░░░░░╚ '''''''''''''''  Author: Tobias Watzel
#    ░░░                     Copyright 2020
#

%matplotlib widget
import cv2
import glob
import re
import io
import ipywidgets as widgets
import numpy as np
from IPython.display import clear_output, Image, display
import PIL.Image
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib as mpl
from matplotlib import cm
from functools import partial
import dill

## Versuchsbeschreibung
Dieser Versuch soll Ihnen ein Gefühl für verschiedene Farbräume geben. Wählen Sie ein Bild aus und lassen Sie sich die verschiedenen Farbkomponenten dieses Bildes anzeigen.

In [2]:
# create dict with name of image and its path (FIX? hacked...)
dict_images = {entry.replace('/',' ').replace('.',' ').split()[1]:entry for entry in sorted(glob.glob("images/*"))}
checkbox_dict = {channel:1 for channel in ['R', 'G', 'B', 'C', 'M', 'Y']}

def convert_image(a, fmt='png'):
    a = np.uint8(np.clip(a, 0, 255))
    f = io.BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    return f.getvalue()
    
    
def update_image():  
    # read image
    image = cv2.imread(dict_images[select.value])
    
    # image in RGB (just switching dims)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_cmy = 255 - cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # set channels for rgb
    for i, channel in enumerate(list('RGB')):
        if not checkbox_dict[channel]:
            image_rgb[:, :, i] = 0
    box.children[1].children[0].value = convert_image(image_rgb)
    
    for i, channel in enumerate(list('CMY')):
        if not checkbox_dict[channel]:
            image_cmy[:, :, i] = 0
    box.children[3].children[0].value = convert_image(255 - image_cmy)

        

init_image = cv2.cvtColor(cv2.imread('images/birne.png'), cv2.COLOR_BGR2RGB)

image_1 = widgets.Image(value=convert_image(init_image), format='png', width=300, height=400)
image_2 = widgets.Image(value=convert_image(init_image), format='png', width=300, height=400)


def change_val(change):
    if change['type'] == 'change' and change['name'] == 'value':
        if len(change['owner'].description) > 1:
            update_image()
        else:
            checkbox_dict[change['owner'].description] = np.int(change['new'])
            update_image()
        
        
    


def create_display(name_colorspace = 'rgb'):
    
    display_list = []
    init_image = cv2.cvtColor(cv2.imread('images/birne.png'), cv2.COLOR_BGR2RGB)
    
    image = widgets.Image(value=convert_image(init_image), format='png', width=300, height=400)
    helper_dict = {}
    for channel in list(name_colorspace):
        helper_dict[channel] = widgets.Checkbox(value=True, description=channel.upper(), disabled=False)
        helper_dict[channel].observe(change_val)
    
    return widgets.HBox([image, widgets.VBox(list(helper_dict.values()))])

# create box
label_1 = widgets.Label(value=r'\(\textbf{RGB (Red, Green, Blue)}\)')
label_2 = widgets.Label(value=r'\(\textbf{CMY (Cyan, Magenta, Yellow)}\)')
box = widgets.VBox([label_1, create_display('rgb'), label_2, create_display('cmy')])

select = widgets.Select(
    options=list(dict_images.keys()),
    value='birne',
    description='Dateiliste:')

select.observe(change_val)

display(select)

display(box)

Select(description='Dateiliste:', options=('birne', 'buch', 'fahrzeug', 'klammermaschine', 'mandrill', 'orange…

### 1. Welche Farbe ergibt sich, wenn man cyan und magenta mischt?


In [3]:
save_mix1 = 'save/mix1.dill'

mix1 = ''
# try to load precicion value
try: 
    with open(save_mix1, 'rb') as fp:
        mix1 = dill.load(fp)
        if mix1 == '':
            mix1 = ''
except:
    pass

text_mix1 = widgets.Text(value=str(mix1),
                              placeholder='',
                              description='',
                              disabled=False,
                              continuous_update=True)

def callback_mix1(change):
    if change['type'] == 'change' and change['name'] == 'value':
        # save when press enter
        with open(save_mix1, 'wb') as fp:
            dill.dump(change['new'], fp)

display(text_mix1)

text_mix1.observe(callback_mix1)

Text(value='', placeholder='')

In [4]:
# Autograding answer, please ignore


### 2. Welche Farbe ergibt sich, wenn man magenta und gelb mischt?

In [6]:
save_mix2 = 'save/mix2.dill'

mix2 = ''
# try to load precicion value
try: 
    with open(save_mix2, 'rb') as fp:
        mix2 = dill.load(fp)
        if mix2 == '':
            mix2 = ''
except:
    pass

text_mix2 = widgets.Text(value=str(mix2),
                              placeholder='',
                              description='',
                              disabled=False,
                              continuous_update=True)

def callback_mix2(change):
    if change['type'] == 'change' and change['name'] == 'value':
        # save when press enter
        with open(save_mix2, 'wb') as fp:
            dill.dump(change['new'], fp)

display(text_mix2)

text_mix2.observe(callback_mix2)

Text(value='Rot', placeholder='')

In [7]:
# Autograding answer, please ignore
